In [1]:
import ipywidgets as widgets
import sys
from pathlib import Path
import os
import importlib


module_path='preprocessing/day_intervals_preproc'
if module_path not in sys.path:
    sys.path.append(module_path)

module_path='utils'
if module_path not in sys.path:
    sys.path.append(module_path)

module_path='preprocessing/hosp_module_preproc'
if module_path not in sys.path:
    sys.path.append(module_path)

module_path='model'
if module_path not in sys.path:
    sys.path.append(module_path)
#print(sys.path)
root_dir = os.path.dirname(os.path.abspath('UserInterface.ipynb'))
import day_intervals_cohort
from day_intervals_cohort import *

import day_intervals_cohort_v2
from day_intervals_cohort_v2 import *

import data_generation_icu

import data_generation
import evaluation

import feature_selection_hosp
from feature_selection_hosp import *

# import train
# from train import *


import ml_models
from ml_models import *

import dl_train
from dl_train import *

import tokenization
from tokenization import *


import behrt_train
from behrt_train import *

import feature_selection_icu
from feature_selection_icu import *
import fairness
import callibrate_output

In [2]:
importlib.reload(day_intervals_cohort)
import day_intervals_cohort
from day_intervals_cohort import *

importlib.reload(day_intervals_cohort_v2)
import day_intervals_cohort_v2
from day_intervals_cohort_v2 import *

importlib.reload(data_generation_icu)
import data_generation_icu
importlib.reload(data_generation)
import data_generation

importlib.reload(feature_selection_hosp)
import feature_selection_hosp
from feature_selection_hosp import *

importlib.reload(feature_selection_icu)
import feature_selection_icu
from feature_selection_icu import *

importlib.reload(tokenization)
import tokenization
from tokenization import *

importlib.reload(ml_models)
import ml_models
from ml_models import *

importlib.reload(dl_train)
import dl_train
from dl_train import *

importlib.reload(behrt_train)
import behrt_train
from behrt_train import *

importlib.reload(fairness)
import fairness

importlib.reload(callibrate_output)
import callibrate_output

importlib.reload(evaluation)
import evaluation

# Welcome to your MIMIC-IV Project

MIMIC-IV v2.0

Please go to:
- https://physionet.org/content/mimiciv/2.0/ for v2.0

Follow instructions to get access to MIMIC-IV dataset.

Download the files using your terminal: 
- wget -r -N -c -np --user mehakg --ask-password https://physionet.org/files/mimiciv/2.0/
        
Save downloaded files in the parent directory of this github repo. 

The structure should look like below for v2.0-
- mimiciv/2.0/hosp
- mimiciv/2.0/icu

In [3]:
version_path="mimiciv/2.0"

# Extract Admissions

Start with 'Mortality' task

#### Mortality Cohort creation

In [4]:
disease_label=""
time=0
label='Mortality'

data_icu=False
data_mort=label=='Mortality'
data_admn=label=='Readmission'
data_los=label=='Length of Stay'

icd_code = 'No Disease Filter'


cohort_output = day_intervals_cohort_v2.extract_data('Non-ICU',label,time,icd_code, root_dir,disease_label)

===========MIMIC-IV v2.0============
EXTRACTING FOR: | NON-ICU | MORTALITY | 0 |


FileNotFoundError: [Errno 2] No such file or directory: '/Users/dfadlon/Documents/mimic/my_mimic_iv_pipline/MIMIC-IV-Data-Pipeline/mimiciv/2.0/hosp/admissions.csv.gz'

### Feature Selection

In [ ]:
diag_flag = False # without diagnosis
lab_flag = True
proc_flag = True
med_flag = True

feature_nonicu(cohort_output, version_path,diag_flag,lab_flag,proc_flag,med_flag)

group_diag = False
group_med = True
group_proc = True

preprocess_features_hosp(cohort_output, diag_flag,proc_flag,med_flag,False,group_diag,group_med,group_proc,False,False,0,0)

Generate Summary

In [ ]:
generate_summary_hosp(diag_flag,proc_flag,med_flag,lab_flag)

Original Pipeline feature selection

In [ ]:
select_diag=False
select_med=True
select_proc=True
select_lab=True
# 'select_out' and 'select_chart' are relevant for ICU
select_out=False
select_chart=False

features_selection_hosp(cohort_output, diag_flag,proc_flag,med_flag,lab_flag,select_diag,select_med,select_proc,select_lab)

Cleaning

In [ ]:

clean_lab=True
impute_outlier=True
thresh=98
left_thresh=0
preprocess_features_hosp(cohort_output, False,False,False,lab_flag,False,False,False,clean_lab,impute_outlier,thresh,left_thresh)

### Data Generation

In [ ]:
agg_method = "forward fill and mean"
impute='Mean'
include=15*24
bucket=4 # interval size
gen=data_generation.Generator(cohort_output,data_mort,data_admn,data_los,diag_flag,lab_flag,proc_flag,med_flag,impute,include,bucket,predW=0)

## Create Heart Failure Cohort

In [ ]:
disease_label="I50"
time=0
label='Readmission'

data_icu=False
data_mort=label=='Mortality'
data_admn=label=='Readmission'
data_los=label=='Length of Stay'

icd_code = 'No Disease Filter'


cohort_output = day_intervals_cohort_v2.extract_data('Non-ICU',label,time,icd_code, root_dir,disease_label)

## My Postprocessing

### Create Final Cohort

- Intersect admissions between heart-failure and mortality
- Filter to real extracted data

In [ ]:
from postprocessing.cohort_intersection_creation import *
from postprocessing.data_post_processing import *

In [ ]:
PATH_TO_MORTALITY_COHORT = ""
PATH_TO_HEART_FAILURE_COHORT = ""

FINAL_COHORT_PATH = "data/cohort/final_cohort.csv.gz"

create_multi_task_cohort(
    data_name_to_path={
        "ihm": PATH_TO_MORTALITY_COHORT,
        "heart_failure": PATH_TO_HEART_FAILURE_COHORT
    },
    output_path=FINAL_COHORT_PATH
)

### One Tabular Data

- Create final one dataframe for each admission
    - Using selected features
    - Concatenate static and dynamic dataframes features
    - Rename columns to features names (instead of codes)

- Concat all admissions data
- Split to chronological datasets (train, valid, test)

- Clean
    - Cross patients (between sets)
    - Empty features (if needed)

- Compose to multi-task label
- Create a numeric version of teh data for traditional ML

In [ ]:
VERSION_NAME = "v1"

os.makedirs("data/processed_data")

post_process_data(
    cohort_path=FINAL_COHORT_PATH,
    version=VERSION_NAME,
    label_plan_key=1,
    output_dir_path="data/processed_data"
)